#### SECTION-1-Data Cleaning and Preparation

###### Installing the Bokeh Library for data visualisation

In [0]:
 %pip install bokeh

Python interpreter will be restarted.
Python interpreter will be restarted.


###### Checking the file path if in File Store

In [0]:
dbutils.fs.rm('/FileStore/tables/clinicaltrial_12019.csv')

Out[1]: False

In [0]:
dbutils.fs.ls('/FileStore/tables')

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/Mohammed_Al_tameemi', name='Mohammed_Al_tameemi', size=1010127, modificationTime=1645533381000),
 FileInfo(path='dbfs:/FileStore/tables/Mohammed_Al_tameemi-1', name='Mohammed_Al_tameemi-1', size=1010127, modificationTime=1645533426000),
 FileInfo(path='dbfs:/FileStore/tables/Mohammed_Al_tameemi-2', name='Mohammed_Al_tameemi-2', size=1010127, modificationTime=1645612637000),
 FileInfo(path='dbfs:/FileStore/tables/accounts-1.zip', name='accounts-1.zip', size=5297592, modificationTime=1645612127000),
 FileInfo(path='dbfs:/FileStore/tables/accounts-2.zip', name='accounts-2.zip', size=5297592, modificationTime=1647446804000),
 FileInfo(path='dbfs:/FileStore/tables/accounts-3.zip', name='accounts-3.zip', size=5297592, modificationTime=1647866139000),
 FileInfo(path='dbfs:/FileStore/tables/accounts-4.zip', name='accounts-4.zip', size=5297592, modificationTime=1647962137000),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip

###### Creating a Re-Usable Codes

In [0]:
import os
my_file_csv = 'clinicaltrial_2021_csv'
my_file_permenant = 'clinicaltrial_2021.csv'
os.environ['my_file_s'] = my_file_csv

###### Copying the file path from the file store to the tmp in shell for un-zipping process

In [0]:
dbutils.fs.cp('/FileStore/tables/'+ my_file_csv + '.gz', 'file:/tmp/')

Out[4]: True

###### Checking the file path in shell

In [0]:
%sh
ls /tmp/

Rserv
Rtmp0Wgt99
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021_csv.gz
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
ipykernel-connection-ReplId-79cfd-e1b9a-c3ac4-a.json
systemd-private-e7e911b8b0504e789a76a450de58be40-apache2.service-SigI2f
systemd-private-e7e911b8b0504e789a76a450de58be40-ntp.service-aeY6Kg
systemd-private-e7e911b8b0504e789a76a450de58be40-systemd-logind.service-2GiCQg
systemd-private-e7e911b8b0504e789a76a450de58be40-systemd-resolved.service-G6k7Wi
tmp.6suM3aTS6Z


###### Unzipping the file using shell command.

In [0]:
%sh 
gunzip /tmp/$my_file_s

###### Checking the file in shell if it's un-zipped

In [0]:
%sh
ls /tmp/

Rserv
Rtmp0Wgt99
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021_csv
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
ipykernel-connection-ReplId-79cfd-e1b9a-c3ac4-a.json
systemd-private-e7e911b8b0504e789a76a450de58be40-apache2.service-SigI2f
systemd-private-e7e911b8b0504e789a76a450de58be40-ntp.service-aeY6Kg
systemd-private-e7e911b8b0504e789a76a450de58be40-systemd-logind.service-2GiCQg
systemd-private-e7e911b8b0504e789a76a450de58be40-systemd-resolved.service-G6k7Wi
tmp.6suM3aTS6Z


###### Moving the zipped file from shell tmp to file store and save it as (.csv)

In [0]:
dbutils.fs.mv('file:/tmp/'+ my_file_csv , '/FileStore/tables/' + my_file_permenant)

Out[8]: True

###### Checking the file name to confirm if it's matching the required name

In [0]:
dbutils.fs.ls('/FileStore/tables/' + my_file_permenant)

Out[9]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021.csv', name='clinicaltrial_2021.csv', size=50359696, modificationTime=1652440680000)]

###### Now we will create an RDD for the Clinicaltraila2021 Dataset

In [0]:
accounts =sc.textFile ("FileStore/tables/" + my_file_permenant)
accounts.take(10)

Out[10]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|',
 'NCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|',
 'NCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate',
 'NCT02752438|Ankara University|Unknown status|May 2016|Jul 

#### SECTION-2- PySpark RDD Implementation

###### Question.1 Finding the number of studies in the dataset.

In [0]:
accounts =sc.textFile ("FileStore/tables/" + my_file_permenant)
the_header = accounts.first()
accounts = accounts.filter(lambda header: header != the_header)
accounts.distinct().count()

Out[11]: 387261

###### Question.2  Listing all the types of studies in the Clinicaltrial Dataset along with the frequencies of each type.

In [0]:
from operator import add
s_accounts=accounts.map(lambda x:x.split('|'))
s_type = s_accounts.map(lambda t: t[5])
s_type = s_type.map(lambda x: (x,1))
s_type = s_type.reduceByKey(add)
s_type= s_type.sortBy(lambda x:(x[1]), ascending = False)
s_type.take(4)

Out[12]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

###### Question.3   Finding the top 5 conditions from the Clinicaltrial Dataset with their frequencies.

In [0]:
s_cond = s_accounts.map(lambda c: c[7])
s_cond = s_cond.flatMap(lambda c: c.split(','))
s_cond = s_cond.filter(lambda c: c != '')
s_cond = s_cond.map(lambda p : (p,1))
s_cond = s_cond.reduceByKey(add)
s_cond = s_cond.sortBy(lambda x:(x[1]), ascending = False)
s_cond.take(5)

Out[13]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

######Question.4  Finding the 5 most frequent roots.

In [0]:
mesh_rdd = sc.textFile('dbfs:/FileStore/tables/mesh.csv')
mesh_split = mesh_rdd.map(lambda s:s.split(',')).map(lambda s:(s[0],s[1].split('.')))
mesh_split = mesh_split.map(lambda d: (d[0],d[1][0]))
mesh_cond = mesh_split.join(s_cond)
mesh_cond = mesh_cond.map(lambda m:m[1])
mesh_cond = mesh_cond.reduceByKey(add)
mesh_cond = mesh_cond.sortBy(lambda x:(x[1]), ascending = False)
mesh_cond.take(5)

Out[14]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

######Question.5   Finding the 10 most common sponsors.

In [0]:
pharma = sc.textFile('dbfs:/FileStore/tables/pharma.csv')
pharma_parent = pharma.map(lambda p: p.split(',')).map(lambda p :p[1]).map(lambda p: (p,1)).reduceByKey(add)
sponsor_rdd = s_accounts.map(lambda t: t[1]).map(lambda t:(t,1)).reduceByKey(add)
parent_sponsor_join = sponsor_rdd.subtractByKey(pharma_parent)
parent_sponsor = parent_sponsor_join.sortBy(lambda j:(j[1]), ascending = False)
parent_sponsor.take(10)

Out[15]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

###### Question.6 Finding the number of completed studies each month in 2021.

In [0]:
import calendar
s_accounts_plot=s_accounts.map(lambda x:(x[2],x[4])).filter(lambda x: x[0]== 'Completed')
s_accounts_plot = s_accounts_plot.map(lambda x: x[1].split(" ")).filter(lambda x: x[-1]== '2021')
s_acc_plot1 = s_accounts_plot.map(lambda e: e[0]).map(lambda r: (r,1)).reduceByKey(add)
completion = {x:y for y,x in enumerate(calendar.month_abbr[1:],1)}
s_acc_plot1 = s_acc_plot1.sortBy(keyfunc=lambda x : completion.get(x[0]))
s_acc_plot1.take(12)

Out[16]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]

#### Data Visualisation (By Using Bokeh Library)

###### Returning the results for the RDD as a Map collection for the Visualisation perpose

In [0]:
s_acc_plot=s_acc_plot1.collectAsMap()

In [0]:

from bokeh.io import output_file,show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral4
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import HoverTool

output_file("colormapped_bars.html")
months = ['','Mar','Oct','']
counts = ['',1227,187,'']
source = ColumnDataSource(data=dict(months=months, counts=counts, color=Spectral4))
plot = figure(x_range=list(s_acc_plot), height=400, sizing_mode="stretch_width", title="COMPLETED_STUDIES_FOR_YEAR_2021",toolbar_location=None, tools="",x_axis_label='MONTHS',y_axis_label='NUMBER_OF_STUDIES')
plot.vbar(x=list(s_acc_plot.keys()), top=list(s_acc_plot.values()), width=0.9,fill_alpha = .7 ,line_alpha = .2, line_color='blue')
plot.vbar(x='months', top='counts', width=0.9, color='color', source=source)
plot.add_tools(HoverTool(tooltips=[("Number_of_Studies","$y")]))
plot.y_range.start = 0
plot.y_range.end = 1400
plot.title.align = "center"
plot.title.text_color = "black"
plot.axis.axis_label_text_color = "black"
plot.title.text_font_size = '14pt'
plot.xaxis.axis_label_text_font_size = "10pt"
plot.axis.axis_label_text_font_style = 'bold'
plot.background_fill_color = "white"
plot.axis.minor_tick_line_color = "Blue"
plot.outline_line_color = "Blue"
html=file_html(plot,CDN)
displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 Bokeh Application